# Imports

In [1]:
import pandas as pd #for analysing and changing data
import networkx as nx #for creating edges and nodes
import IPython #for showing html output in cells
from pyecharts.charts import Graph #library for visualising Network
from pyecharts import options as opts #further visualising options

# Reading data

In [2]:
df = pd.read_csv("my-edgy-friends.edges",names = ["node1", "node2", "value"] ) #reading Data into pandas DataFrame adjusting columns

In [3]:
df.drop(index=0, inplace=True) #drop old header

In [5]:
df.head(2) #showing first two rows of Data

,node1,node2,value
1,Diana Ivanova,Berlin4Future #BidenForParis,2
2,Diana Ivanova,Marie von den Benken,2


In [6]:
print(f'The Datframe has the dimension {df.shape[0]} rows and {df.shape[1]} columns')

The Datframe has the dimension 433 rows and 3 columns


That means we have an edgelist of 433 edges, so lets look at the nodes when we create the graph

# Create our Network

### Initiate our Network Graph Object

In [53]:
G = nx.Graph()

### Bringing the edges in the graph

In [54]:
def create_edgelist(df: pd.DataFrame) -> [(str, str)]:
    """
    takes:
        a pandas datframe with target and source nodes in columsn
    returns:
        an edgelist in form of [(source, target)]

    """
    edgelist = [] #initiate empty edgelist
    for index, row in df.iterrows(): #iterating over the whole datframe
        edgelist.append([str(row.node1), str(row.node2)]) # edgelist is a list of tuples containing source and target
    return edgelist


In [55]:
edgelist = create_edgelist(df)

In [56]:
G.add_edges_from(edgelist) #bringing the edges in our networkx graph object

In [57]:
G.nodes()#take a look at the nodes

NodeView(('Diana Ivanova', 'Berlin4Future #BidenForParis', 'Marie von den Benken', 'Mai Thi Nguyen-Kim', 'Tommi Schmitt', 'Scientists for Future', 'Luisa Neubauer', 'DevelopersForFuture  #WeVsClimateCrisis', 'Rezo', 'Ende Gelnde', 'Karl Lauterbach', 'Fridays For Future Germany', 'Nico Semsrott', 'Fabian Kster', 'Hazel Brugger', 'Volksverpetzer', 'Bill Gates', 'Ole', 'teresa bcker', 'extra3', 'Juju', 'Felix Lobrecht', 'Till Reiners', 'erzaehlmirnix', 'Sven Stueven', 'Der Postillon', 'Tobse', 'Schdegie.', 'L. Duy Pham', 'TheMinnieTheMouse', 'Barack Obama', 'Elon Musk', 'ESL', 'INA HOUT', 'ZDF heute-show', 'Joyce', "Let'sPlayBros"))

Its not necessary to add nodes manually, since the <code>networkx<code/> automatically filters out unique nodes

In [58]:
centrality = nx.degree_centrality(G) #calculating the centrality degree of every node in the network
centrality # having a look at the centrality degrees

{'Diana Ivanova': 0.5277777777777778,
 'Berlin4Future #BidenForParis': 0.75,
 'Marie von den Benken': 0.6388888888888888,
 'Mai Thi Nguyen-Kim': 0.9166666666666666,
 'Tommi Schmitt': 0.861111111111111,
 'Scientists for Future': 0.75,
 'Luisa Neubauer': 0.8888888888888888,
 'DevelopersForFuture  #WeVsClimateCrisis': 0.861111111111111,
 'Rezo': 0.8888888888888888,
 'Ende Gelnde': 0.7222222222222222,
 'Karl Lauterbach': 0.8333333333333333,
 'Fridays For Future Germany': 0.8333333333333333,
 'Nico Semsrott': 0.8055555555555555,
 'Fabian Kster': 0.8333333333333333,
 'Hazel Brugger': 0.75,
 'Volksverpetzer': 0.75,
 'Bill Gates': 0.5,
 'Ole': 0.9444444444444444,
 'teresa bcker': 0.8888888888888888,
 'extra3': 0.9444444444444444,
 'Juju': 0.6944444444444444,
 'Felix Lobrecht': 0.75,
 'Till Reiners': 0.6666666666666666,
 'erzaehlmirnix': 0.611111111111111,
 'Sven Stueven': 0.41666666666666663,
 'Der Postillon': 0.7222222222222222,
 'Tobse': 0.8055555555555555,
 'Schdegie.': 0.75,
 'L. Duy Pham'

In [59]:
centrality_top5= sorted(centrality, key=centrality.get, reverse=True)[:5] #making a list of the top 5 nodes regarding degree centrality
print(f"the top 5 nodes are : {centrality_top5} ")

the top 5 nodes are : ['Ole', 'extra3', 'Mai Thi Nguyen-Kim', 'Luisa Neubauer', 'Rezo'] 


# Visualisation

In [60]:
def configure_nodes_visualisation(G : nx.Graph)->[dict]:
    """
    takes:
        an networkx graph object
    returns:
        a list of dicts suitable for pyecharts network  visualisation nodes
    """
    centrality = nx.degree_centrality(G) #calculating the centrality degree of every node in the network
    centrality_top5= sorted(centrality, key=centrality.get, reverse=True)[:5] #making a list of the top 5 nodes regarding degree centrality
    nodes = []
    for node in G.nodes():
        if centrality[node]> 0.001: #try to minimize the size of tghe network, helpfull for large networks
            if node in centrality_top5: #make another layout for top5 nodes
                nodes.append({"name": node, "symbol": "triangle", "symbolSize": centrality[node]*10 , "categorie": "top5", "draggable":True,"itemStyle": {
                                    "color": "#ff3f76"}, "label": {"show": True}, "value":round(centrality[node],2)})
            # symbol as triagle , size depending on centrality degree, label is shown
            else:
                nodes.append({"name": node, "symbol": "rect", "symbolSize": centrality[node]*10, "categorie": "ordinary","draggable":True, "itemStyle": {
                                    "color": "#789704"}, "label": {"show": centrality[node]>0.74}, "value":round(centrality[node], 2)}) #symbol as triagle , size depending on centrality degree, label is shown if centrality degree ist over 0.74
    return nodes #return the list of nodes

In [61]:
nodes = configure_nodes_visualisation(G) #configuring the nodes of the graph object

In [62]:
nodes[:2] #example of the created list

[{'name': 'Diana Ivanova',
  'symbol': 'rect',
  'symbolSize': 5.277777777777778,
  'categorie': 'ordinary',
  'draggable': True,
  'itemStyle': {'color': '#789704'},
  'label': {'show': False},
  'value': 0.53},
 {'name': 'Berlin4Future #BidenForParis',
  'symbol': 'rect',
  'symbolSize': 7.5,
  'categorie': 'ordinary',
  'draggable': True,
  'itemStyle': {'color': '#789704'},
  'label': {'show': True},
  'value': 0.75}]

In [63]:
def configure_edges_visualisation(G:nx.Graph)->[dict]:
    """
    takes: 
        an networkx graph object
    returns:
        a list of dicts suitable for pyecharts network  visualisation edges
    """
    links = []
    for index, row in df.iterrows(): #iterating over whole datframe
        links.append({"source": str(row.node1), "target": str(row.node2), "value": int(row.value)}) #adding the columsn into a dict
    
    return links

In [64]:
links = configure_edges_visualisation(G)

In [65]:
links[:2]#example of two edge dicts 

[{'source': 'Diana Ivanova',
  'target': 'Berlin4Future #BidenForParis',
  'value': 2},
 {'source': 'Diana Ivanova', 'target': 'Marie von den Benken', 'value': 2}]

## Create the visualisation

In [79]:
fig = Graph() #create figure object
fig.height = "900px"
fig.width = "900px"
fig.add("Tobis Twitter Network",nodes, links,tooltip_opts = "{a} <br> {b} : {c}", repulsion=0, gravity= 1, linestyle_opts = opts.LineStyleOpts(width= 0.1, curve= 0.1, color="source")) #adding nodes and edges (called links) into the figure format layout, set labels for tooltip

## Showing the Result
### zoom in to take a better look!


In [80]:
fig.render_notebook()

# Aditional Inside of Friends of Friends Network

In [68]:
df2 = pd.read_csv("my-edgy-friend-network.edges", names = ["node1", "node2", "value"])
df2.drop(index=0, inplace=True) #drop old header
df2 = df2.fillna(1)

In [69]:
edgelist2 = create_edgelist(df2)

In [70]:
G2 = nx.Graph()
G2.add_edges_from(edgelist2)


In [73]:
centrality2 = nx.degree_centrality(G2) #calculating the centrality degree of every node in the network

In [81]:
centrality_top20= sorted(centrality2, key=centrality2.get, reverse=True)[:20] #making a list of the top 5 nodes regarding degree centrality
print(f"the top 20 nodes are : {centrality_top20} ")

the top 20 nodes are : ['813286', '16745492', '15426691', '1277617463400828928', '1068337838', '2361507014', '1094849342', '3232960847', '1103391316464058369', '1072187272815149057', '3165843875', '816571915057897472', '1112222741548224512', '825306344961998848', '894965941066510336', '633100156', '3683359935', '16905329', '3292982985', '50393960'] 
